In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('all_assigned_PISCES.txt', delim_whitespace=True, header=None, low_memory=False) #importing the file containing all the assignments for the PDB IDs after being concatenated.
df.columns =['ASSIGNMENT', 'RESIDUE', 'CHAIN', 'RESIDUE_NUMBER', 'GIVEN_NUMBER', 'SS_CODE', 'SS_NAME', 'X_CORD', 'Y_CORD', 'Z_CORD', 'PDB_ID'] #naming headers

In [20]:
df1=df[['RESIDUE','SS_NAME']] #taking out the required columns
df1

,RESIDUE,SS_NAME
0,ALA,Coil
1,GLY,Coil
2,GLU,Coil
3,ASP,Coil
4,VAL,Coil
...,...,...
1995500,ALA,AlphaHelix
1995501,GLY,AlphaHelix
1995502,PHE,AlphaHelix
1995503,SER,AlphaHelix


In [21]:
amino_acid_freq=df1['RESIDUE'].value_counts() #calculating the counts of each amino acids in the given data
amino_acid_freq=amino_acid_freq.to_frame() #changing to dataframe
amino_acid_freq

,RESIDUE
LEU,186433
ALA,165819
GLY,143486
VAL,138933
GLU,131556
SER,121277
ASP,120048
ILE,113679
LYS,111545
THR,109911


In [22]:
total_res=amino_acid_freq['RESIDUE'].sum() #calculating the total number of residues
total_res

1995505

In [23]:
ss_freq=df1['SS_NAME'].value_counts() #calculating the counts of residues occuring in each secondary structure type
ss_freq=ss_freq.to_frame()
ss_freq

,SS_NAME
AlphaHelix,688574
Strand,452701
Turn,392657
Coil,355622
310Helix,83635
Bridge,22040
PiHelix,255


In [24]:
ss_freq1=pd.DataFrame(index=['Helix', 'Strand', 'Coil'], columns=['SS_NAME']) #creating an empty dataframe
ss_freq1.loc['Helix','SS_NAME']=ss_freq.loc['AlphaHelix','SS_NAME']+ss_freq.loc['310Helix','SS_NAME'] #+ss_freq.loc['PiHelix','SS_NAME']
ss_freq1.loc['Strand','SS_NAME']=ss_freq.loc['Strand','SS_NAME']+ss_freq.loc['Bridge','SS_NAME']
ss_freq1.loc['Coil','SS_NAME']=ss_freq.loc['Coil','SS_NAME']+ss_freq.loc['Turn','SS_NAME']
ss_freq1

,SS_NAME
Helix,772209
Strand,474741
Coil,748279


In [25]:
amino_acid_ss_freq= df1.groupby(['RESIDUE','SS_NAME']).size() #calculating the counts of each amino acid occuring in each secondary structure type
amino_acid_ss_freq=amino_acid_ss_freq.to_frame() #converting to dataframe
filename='amino_acid_ss_freq_3_state.csv'
amino_acid_ss_freq.to_csv(filename) #saving to csv file
print('dataframe is written to csv file successfully')

dataframe is written to csv file successfully


In [26]:
amino_acid_ss_mapping = pd.read_csv('amino_acid_ss_freq_3_state.csv')
dff=amino_acid_ss_mapping #don't know why I did this but don't want to change it now because the code is working

In [27]:
dff


,RESIDUE,SS_NAME,0
0,ALA,310Helix,7667
1,ALA,AlphaHelix,80915
2,ALA,Bridge,1173
3,ALA,Coil,21220
4,ALA,PiHelix,12
...,...,...,...
144,VAL,Bridge,1774
145,VAL,Coil,18267
146,VAL,PiHelix,20
147,VAL,Strand,58443


In [28]:
amino_acids=list(dff.RESIDUE.unique()) #making list of amino acids

In [29]:
ss_list=list(dff.SS_NAME.unique()) #making list of secondary structures

In [30]:
aa_vs_ss_table= pd.DataFrame(index=amino_acids, columns=['Helix', 'Strand', 'Coil']) #creating an empty dataframe
ast=aa_vs_ss_table
ast

,Helix,Strand,Coil
ALA,NaN,NaN,NaN
ARG,NaN,NaN,NaN
ASN,NaN,NaN,NaN
ASP,NaN,NaN,NaN
CYS,NaN,NaN,NaN
GLN,NaN,NaN,NaN
GLU,NaN,NaN,NaN
GLY,NaN,NaN,NaN
HIS,NaN,NaN,NaN
ILE,NaN,NaN,NaN


In [31]:
#simpler count calculation; subset of propensity calculation
#for helix, j is the amino acid
for j in amino_acids:
    i='Helix'
    try:
        a=list(dff.loc[(dff['RESIDUE']==j) & (dff['SS_NAME']=='310Helix')]['0'])
        if len(a)==0:
            a=[0]

        b=list(dff.loc[(dff['RESIDUE']==j) & (dff['SS_NAME']=='AlphaHelix')]['0'])
        if len(b)==0:
            b=[0]
        
        c=list(dff.loc[(dff['RESIDUE']==j) & (dff['SS_NAME']=='PiHelix')]['0'])
        if len(c)==0:
            b=[0]
            
        y=int(a[0])+int(b[0])+int(c[0]) 
        ast.loc[j,i]=y
    except IndexError:
        ast.loc[j,i]=0
        
#for strand, j is the amino acid
for j in amino_acids:
    i='Strand'
    try:
        a=list(dff.loc[(dff['RESIDUE']==j) & (dff['SS_NAME']=='Bridge')]['0'])
        if len(a)==0:
            a=[0]


        b=list(dff.loc[(dff['RESIDUE']==j) & (dff['SS_NAME']=='Strand')]['0'])
        if len(b)==0:
            b=[0]
        
        y=int(a[0])+int(b[0]) 
        ast.loc[j,i]=y
    except IndexError:
        ast.loc[j,i]=0



#for coil, j is the amino acid
for j in amino_acids:
    i='Coil'
    try:
        a=list(dff.loc[(dff['RESIDUE']==j) & (dff['SS_NAME']=='Coil')]['0'])
        if len(a)==0:
            a=[0]
            
        b=list(dff.loc[(dff['RESIDUE']==j) & (dff['SS_NAME']=='Turn')]['0'])
        if len(b)==0:
            b=[0]
        
        y=int(a[0])+int(b[0]) 
        ast.loc[j,i]=y
    except IndexError:
        ast.loc[j,i]=0

aa_vs_ss_table=ast       
aa_vs_ss_table
filename='aa_vs_ss_3_state.csv'
aa_vs_ss_table.to_csv(filename) #saving to csv file

In [32]:
propensity_table= pd.DataFrame(index=amino_acids, columns=['Helix', 'Strand', 'Coil']) #making an empty dataframe for propensity value calculation
pt=propensity_table

In [33]:

for i in ast.columns:
    for j in ast.index: #looping to each cell
        try:
            aa_ss=ast.loc[j,i]
            aa_count=amino_acid_freq.loc[j]['RESIDUE'] #count of amino acid j in all proteins
            ss_count=ss_freq1.loc[i]['SS_NAME'] #count of secondary structure type i in all proteins
            propensity_aa_ss= (aa_ss/aa_count)/(ss_count/total_res) #calculating propensitites from https://www.sciencedirect.com/science/article/pii/S0006291X06002543?via%3Dihub
            pt.loc[j,i]=propensity_aa_ss #inputting values
#             print(j,i,y[0])
        except IndexError: #to handle error where count of amino acid in said secondary structure is zero
            pt.loc[j,i]=0
pt

filename='propensity_3_state.csv'
pt.to_csv(filename) #saving to csv file
